# Visualise distributions for paper

## Setup

In [1]:
# Hack to emulate running notebook from root directory.
import os
os.chdir('..')

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as pd

from prob_models import prior_model, posterior_model

from utils.plotting import lighten_color
from utils.plotting import init_profile_fig, add_profile

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [3]:
from configs.experiment_handling import get_experiment_config

In [4]:
expt_settings, base_params = get_experiment_config('base')
dataset_dir = os.path.join(*expt_settings['dataset_dir'])
prob_settings = expt_settings['probability_settings']

## Plot distributions of generation data

In [5]:
years = [2010,2015,2019]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
dark_colors = [mcolors.rgb2hex(plt.cm.tab20b.colors[0])] + [mcolors.rgb2hex(c) for c in plt.cm.Dark2.colors]
lighten = [1,0.6,1.3]

period = 24*10+1
starts = [24*31,24*185]
seasons = ['winter','summer']

In [ ]:
for offset,season in zip(starts,seasons):
    fig,ax = plt.subplots(figsize=(15,5))

    for (i,tech) in enumerate(['wind','solar']):
        for (j,year) in enumerate(years):

            data = pd.read_csv(os.path.join(dataset_dir,tech,f'{year}.csv'))

            start = pd.to_datetime("01/01/2000") + pd.DateOffset(hours=offset)
            trange = pd.date_range(start=start, periods=period, freq='h')

            plt.plot(
                trange,
                data[offset:offset+period],
                label=f'{year}',
                alpha=0.75,
                color=lighten_color(colors[i],lighten[j]),
                zorder=10-i
            )

    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %d'))
    plt.xlim(trange[0],trange[-1])
    plt.margins(y=0.2)
    plt.ylim(0,1.1)
    plt.ylabel('Normalized Energy Generation')
    plt.legend(title="Wind      Solar",loc='upper left', ncol=2, labelspacing=0.4, columnspacing=0.8, fontsize=8)
    plt.tight_layout()
    plt.savefig(os.path.join('plots',f'gen_data_{season}.pdf'))
    plt.show()

Plot interactive version with full set of data

In [7]:
dashes = ['solid','dash','dashdot','dot']

In [ ]:
fig = init_profile_fig(y_titles={'y1':'Normalized Energy Generation'})

for (i,tech) in enumerate(['wind','solar']):
    for (j,year) in enumerate(expt_settings[f'{tech}_years']):

        data = pd.read_csv(os.path.join(dataset_dir,tech,f'{year}.csv')).to_numpy().flatten()

        fig = add_profile(
            fig,
            data,
            name=year,
            legendgroup=tech,
            legendgrouptitle_text=f"{tech}".title(),
            line=dict(
                color=f'rgb{lighten_color(colors[i],lighten[j%3])}',
                width=3,
                dash=dashes[j//3]
            ),
            visible=True if j<3 else 'legendonly',
            hovertemplate ='Time: %{x}'+'<br>Norm gen.: %{y:.3f}',
        )

fig.update_layout(
    legend=dict(
        title="Load year",
        orientation="v",
        x=1,
        xanchor='center',
        y=1,
        yanchor='top',
        groupclick="toggleitem"
    )
)

fig['layout']['xaxis'].update(range=['2000-04-24','2000-05-04'])
fig.update_layout(title=dict(
    text='Normalized renewable generation dataset',
    y=0.95,
    x=0.5,
    xanchor='center',
    yanchor='top'
))
fig.write_html(f'generation_dataset_plot.html')

fig.show()

## Plot example prior and posterior distribution for a storage parameter

Pick VRBF cost as variance is high

In [9]:
np.random.seed(0)

In [10]:
theta_scenarios, z_scenarios = prior_model(
    prob_settings,
    base_params['cost_values'],
    base_params['timeseries_values'],
    base_params['storage_values'],
    n_samples=prob_settings['n_prior_samples']
)

In [11]:
j=1
vartheta_scenarios = posterior_model(
            z_scenarios[j],
            prob_settings,
            n_samples=prob_settings['n_posterior_samples']
        )

In [12]:
tech = 'VRBF'

In [ ]:
fig, ax = plt.subplots()

sns.kdeplot(
    [scen.storage_efficiencies[tech] for scen in theta_scenarios],
    cut=0,
    label='Prior',
    ax=ax, c='k', lw=2
)
sns.kdeplot(
    [scen.storage_efficiencies[tech] for scen in theta_scenarios],
    alpha=0.15,
    ax=ax, c='k', lw=2
)
ymax = ax.get_ylim()[1]*1.5
plt.vlines(
    z_scenarios[j].storage_efficiencies[tech],
    0,
    ymax,
    label='Measurement',
    color='k', alpha=0.7, ls='--', lw=2
)
xlims = ax.get_xlim()
plt.plot(0, 0, c='k', alpha=0.5, lw=2, label='Posterior')
plt.legend(loc='upper left')

ax2 = ax.twinx()
sns.kdeplot(
    [scen.storage_efficiencies[tech] for scen in vartheta_scenarios],
    cut=0,
    label='Posterior',
    ax=ax2, c='k', alpha=0.5, lw=2
)
sns.kdeplot(
    [scen.storage_efficiencies[tech] for scen in vartheta_scenarios],
    alpha=0.15,
    ax=ax2, c='k', lw=2
)

#ax.set_xlim(xlims)
ax.set_xlim(0.625,0.875)
ax.set_ylim(0,ymax)
ax.get_yaxis().set_ticks([])
ax.yaxis.set_label_text('Scaled Density')
ax2.get_yaxis().set_ticks([])
ax2.yaxis.set_label_text('')
#ax.set_xlabel(f'{tech} Round-trip Efficiency')
ax.set_xlabel(f'VRFB Round-trip Efficiency')
plt.tight_layout()
plt.savefig(os.path.join('plots','measurement_dists.pdf'))
plt.show()

Note, the densities are rescaled to make the figure look better proportioned, not they have have much meaning anyways